# Module 3 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## k Nearest Neighbors and Model Evaluation

In this programming assignment you will use k Nearest Neighbors (kNN) to build a "model" that will estimate the compressive strength of various types of concrete. This assignment has several objectives:

1. Implement the kNN algorithm with k=9. Remember...the data + distance function is the model in kNN. In addition to asserts that unit test your code, you should "test drive" the model, showing output that a non-technical person could interpret.

2. You are going to compare the kNN model above against the baseline model described in the course notes (the mean of the training set's target variable). You should use 10 fold cross validation and Mean Squared Error (MSE):

$$MSE = \frac{1}{n}\sum^n_i (y_i - \hat{y}_i)^2$$

as the evaluation metric ("error"). Refer to the course notes for the format your output should take. Don't forget a discussion of the results.

3. use validation curves to tune a *hyperparameter* of the model. 
In this case, the hyperparameter is *k*, the number of neighbors. Don't forget a discussion of the results.

4. evaluate the *generalization error* of the new model.
Because you may have just created a new, better model, you need a sense of its generalization error, calculate that. Again, what would you like to see as output here? Refer to the course notes. Don't forget a discussion of the results. Did the new model do better than either model in Q2?

5. pick one of the "Choose Your Own Adventure" options.

Refer to the "course notes" for this module for most of this assignment.
Anytime you just need test/train split, use fold index 0 for the test set and the remainder as the training set.
Discuss any results.

## Load the Data

The function `parse_data` loads the data from the specified file and returns a List of Lists. The outer List is the data set and each element (List) is a specific observation. Each value of an observation is for a particular measurement. This is what we mean by "tidy" data.

The function also returns the *shuffled* data because the data might have been collected in a particular order that *might* bias training.

In [1]:
import random
import statistics
import scipy
from typing import List, Dict, Tuple, Callable

In [2]:
def parse_data(file_name: str) -> List[List]:
    data = []
    file = open(file_name, "r")
    for line in file:
        datum = [float(value) for value in line.rstrip().split(",")]
        data.append(datum)
    random.shuffle(data)
    return data

In [3]:
data = parse_data("concrete_compressive_strength.csv")

In [4]:
data[0]

[366.0, 187.0, 0.0, 191.3, 6.6, 824.3, 756.9, 28.0, 65.91]

In [5]:
len(data)

1030

There are 1,030 observations and each observation has 8 measurements. The data dictionary for this data set tells us the definitions of the individual variables (columns/indices):

| Index | Variable | Definition |
|-------|----------|------------|
| 0     | cement   | kg in a cubic meter mixture |
| 1     | slag     | kg in a cubic meter mixture |
| 2     | ash      | kg in a cubic meter mixture |
| 3     | water    | kg in a cubic meter mixture |
| 4     | superplasticizer | kg in a cubic meter mixture |
| 5     | coarse aggregate | kg in a cubic meter mixture |
| 6     | fine aggregate | kg in a cubic meter mixture |
| 7     | age | days |
| 8     | concrete compressive strength | MPa |

The target ("y") variable is a Index 8, concrete compressive strength in (Mega?) [Pascals](https://en.wikipedia.org/wiki/Pascal_(unit)).

## Train/Test Splits - n folds

With n fold cross validation, we divide our data set into n subgroups called "folds" and then use those folds for training and testing. You pick n based on the size of your data set. If you have a small data set--100 observations--and you used n=10, each fold would only have 10 observations. That's probably too small. You want at least 30. At the other extreme, we generally don't use n > 10.

With 1,030 observations, n = 10 is fine so we will have 10 folds.
`create_folds` will take a list (xs) and split it into `n` equal folds with each fold containing one-tenth of the observations.

In [6]:
def create_folds(xs: List, n: int) -> List[List[List]]:
    k, m = divmod(len(xs), n)
    # be careful of generators...
    return list(xs[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [7]:
folds = create_folds(data, 10)

In [8]:
len(folds)

10

We always use one of the n folds as a test set (and, sometimes, one of the folds as a *pruning* set but not for kNN), and the remaining folds as a training set.
We need a function that'll take our n folds and return the train and test sets:

In [9]:
def create_train_test(folds: List[List[List]], index: int) -> Tuple[List[List], List[List]]:
    training = []
    test = []
    for i, fold in enumerate(folds):
        if i == index:
            test = fold
        else:
            training = training + fold
    return training, test

We can test the function to give us a train and test datasets where the test set is the fold at index 0:

In [10]:
train, test = create_train_test(folds, 0)

In [11]:
len(train)

927

In [12]:
len(test)

103

## Answers

Answer the questions above in the space provided below, adding cells as you need to.
Put everything in the helper functions and document them.
Document everything (what you're doing and why).
If you're not sure what format the output should take, refer to the course notes and what they do for that particular topic/algorithm.

## Problem 1: kNN

Implement k Nearest Neighbors with k = 9.

<a id="distance"></a>
## distance

Calculates distance using a modified euclidean distance (euclidean distance without the final square root) for each index in the provided lists. Lists must be equal length.

Variables
* **p1** List: values to use for calculating distance
* **p2** List: second set of values to use for calculating distance

**returns** float: the sum of every distance between all values in the lists

In [13]:
def distance(p1: List, p2: List):
    return sum([(p1[i]-p2[i])**2 for i in range(len(p1))])

In [14]:
assert distance([0,0], [1,1]) == 2
assert distance([0,0,0], [2,2,1.5]) == 10.25
assert distance([1,1.5,0.2], [5,1,0.4]) == 16.29

<a id="find_neighbors"></a>
## find_neighbors

Finds k-nearest neighbors for a specified index from the test data in the training data.

Variables
* **training_data** List[List]: List of Lists that are the concrete information - should be the training set
* **test_data** List[List]: List of Lists that are the concrete information - should be the training set
* **test_index** int: the index to get the test_row from in the test_data
* **k** int: the number of nearest neighbors to return

**returns** List[Tuple(List, float)]: the list of tuples that has a single row of training data and the distance to the test_row

In [15]:
def find_neighbors(training_data: List[List], test_data: List[List], test_index: int, k: int):
    distances = []
    for training_row in training_data:
        distances.append((distance(training_row, test_data[test_index]), training_row))
    distances.sort(key=lambda tup: tup[0])
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i])
    return neighbors

In [16]:
t1_training_data = [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]
t2_training_data = [[1,1,1,1],[3,3,3,3],[5,5,5,5],[31,31,31,33],[5,5,5,333]]
t1_test_data = [[2,2,2,2],[3,3,3,3],[4,4,4,4],[5,5,5,5]]
t2_test_data = [[2,2,2,2],[3,3,3,3],[4,4,4,4],[44,44,44,44]]
assert find_neighbors(t1_training_data, t1_test_data, 2, 2) == [(0, [4, 4, 4, 4]), (4, [3, 3, 3, 3])]
assert find_neighbors(t2_training_data, t1_test_data, 3, 2) == [(0, [5, 5, 5, 5]), (16, [3, 3, 3, 3])]
assert find_neighbors(t1_training_data, t2_test_data, 3, 2) == [(6400, [4, 4, 4, 4]), (6724, [3, 3, 3, 3])]
assert find_neighbors(t2_training_data, t2_test_data, 3, 2) == [(628, [31, 31, 31, 33]), (6084, [5, 5, 5, 5])]

In [17]:
k = 9
test_index = 35
test_row = test[test_index]
nearest_neighbors = find_neighbors(train, test, test_index, k)
output = 'Row with index ' + str(test_index) + ' being tested: ' + ', '.join([str(e) for e in test_row])
for i in range(len(nearest_neighbors)):
    output += '\r\nclosest neighbor rank ' + str(i + 1) + ': distance=' + str(round(nearest_neighbors[i][0], 2)) + ', Row: ' + ', '.join([str(e) for e in nearest_neighbors[i][1]])
print(output)

Row with index 35 being tested: 290.4, 0.0, 96.2, 168.1, 9.4, 961.2, 865.0, 3.0, 22.5
closest neighbor rank 1: distance=111.31, Row: 295.7, 0.0, 95.6, 171.5, 8.9, 955.1, 859.2, 3.0, 22.95
closest neighbor rank 2: distance=269.11, Row: 290.4, 0.0, 96.2, 168.1, 9.4, 961.2, 865.0, 14.0, 34.67
closest neighbor rank 3: distance=394.16, Row: 295.7, 0.0, 95.6, 171.5, 8.9, 955.1, 859.2, 14.0, 35.23
closest neighbor rank 4: distance=774.82, Row: 290.4, 0.0, 96.2, 168.1, 9.4, 961.2, 865.0, 28.0, 34.74
closest neighbor rank 5: distance=1010.96, Row: 277.1, 0.0, 97.4, 160.6, 11.8, 973.9, 875.6, 14.0, 41.89
closest neighbor rank 6: distance=1040.26, Row: 295.7, 0.0, 95.6, 171.5, 8.9, 955.1, 859.2, 28.0, 39.94
closest neighbor rank 7: distance=1803.6, Row: 277.1, 0.0, 97.4, 160.6, 11.8, 973.9, 875.6, 28.0, 48.28
closest neighbor rank 8: distance=2128.38, Row: 250.0, 0.0, 95.7, 187.4, 5.5, 956.9, 861.2, 3.0, 13.82
closest neighbor rank 9: distance=2179.9, Row: 250.0, 0.0, 95.7, 187.4, 5.5, 956.9, 861

## Problem 2: Evaluation vs. The Mean

Using Mean Squared Error (MSE) as your evaluation metric, evaluate your implement above and the Null model, the mean.

rotate through using 10 folds
calculate the 'y' for 10 random test points in each fold instance
    in those calculations, get the MSE of the prediction vs the actual
get the average of these MSE
then get the standard deviation of it

<a id="mse"></a>
## mse

Calculates Mean Squared Error (MSE) -- Lists must be equal length since it comparing point to point.

Variables
* **y** List: values to use for calculating distance
* **yh** List: second set of values to use for calculating distance

**returns** float: MSE of the list compared to list

In [18]:
def mse(y: List, yh: List):
    return (sum([(y[i]-yh[i])**2 for i in range(len(y))]))/len(y)

In [19]:
assert mse([0,0], [1,1]) == 1
assert mse([0,0,0], [2,2,1.5]) == 10.25/3
assert mse([1,1.5,0.2], [5,1,0.4]) == 16.29/3

<a id="evaluate_model"></a>
## evaluate_model

Evaluates the model by running a 10 fold cross validation and MSE. We will find the average error and the standard deviation.

Variables
* **folds** List[List[List]]: list of lists of lists.... basically this is the dataset broken up into n-many folds
* **k** int: this is the value of k to use for finding neighbors
* **should_print_full** bool: this will determine whether to print out full details or just the final MSE and Std dev values

**returns** float, float: this prints out the metrics if ou tell it to, and it also returns the avg_mse and the standard deviation of that avg_mse

In [20]:
def evaluate_model(evaluation_folds: List[List[List]], k: int, should_print_full):
    mses = []
    for i in range(len(evaluation_folds)):
        fold_results, y, yh = [], [], []
        train, test = create_train_test(evaluation_folds, i)
        for n in range(10):
            random_index = random.randint(0, len(test) - 1)
            nearest_neighbors = find_neighbors(train, test, random_index, k)
            y.append(test[random_index][8])
            yh.append(nearest_neighbors[0][1][8])
            fold_results.append((i, n, random_index, test[random_index][8], nearest_neighbors[0][1][8], test[random_index], nearest_neighbors))
        error = round(mse(y, yh), 6)
        if should_print_full:
            print('\r\nFold ' + str(i) + ' Results:\r\nMSE Error: ' + str(error))
            print('Y (real) values:       ' + ', '.join([str(x) for x in y]))
            print('Yh (predicted) values: ' + ', '.join([str(x) for x in yh]))
        mses.append(error)
    avg_error = sum(mses)/len(mses)
    if should_print_full:
        print('\r\nAverage MSE: ' + str(round(avg_error, 6)))
        print('MSE Standard Deviation: ' + str(round(statistics.stdev(mses, avg_error), 6)))
    return str(round(avg_error, 6)), str(round(statistics.stdev(mses, avg_error), 6))

In [21]:
evaluate_model(folds, 9, True)


Fold 0 Results:
MSE Error: 8.61554
Y (real) values:       60.2, 40.93, 24.0, 71.3, 40.56, 39.42, 40.15, 15.82, 13.36, 55.9
Yh (predicted) values: 56.7, 40.93, 24.0, 71.3, 36.45, 39.42, 43.58, 15.52, 20.08, 55.9

Fold 1 Results:
MSE Error: 66.8507
Y (real) values:       46.2, 26.97, 46.2, 44.09, 63.4, 40.29, 57.21, 49.2, 41.94, 40.29
Yh (predicted) values: 34.4, 29.87, 34.4, 36.94, 77.3, 32.11, 57.22, 49.2, 43.8, 32.11

Fold 2 Results:
MSE Error: 68.26273
Y (real) values:       56.14, 33.66, 48.59, 40.23, 30.57, 71.99, 49.77, 35.34, 47.13, 56.7
Yh (predicted) values: 55.25, 16.89, 51.72, 45.71, 17.54, 78.8, 49.77, 38.56, 50.77, 45.7

Fold 3 Results:
MSE Error: 37.13487
Y (real) values:       52.44, 35.23, 61.24, 39.29, 26.15, 61.24, 56.34, 26.91, 39.61, 21.07
Yh (predicted) values: 52.45, 35.23, 61.23, 38.6, 26.14, 61.23, 47.97, 15.69, 46.23, 9.62

Fold 4 Results:
MSE Error: 53.77536
Y (real) values:       29.0, 32.53, 2.33, 47.74, 32.72, 37.92, 47.81, 13.12, 46.25, 44.3
Yh (predicted)

('50.811413', '24.739506')

## Problem 3: Hyperparameter Tuning

Tune the value of k.

In [22]:
for i in range(1, 20):
    print('\r\nResults for k = ' + str(i))
    errors = []
    stdvs = []
    for n in range(20):
        avg_error, stdv = evaluate_model(folds, i, False)
        errors.append(float(avg_error))
        stdvs.append(float(stdv))
    avg_e = round(sum(errors)/len(errors), 4)
    avg_s = round(sum(stdvs)/len(stdvs), 4)
    print('Average MSEs: ' + str(avg_e))
    print('Average Stdev: ' + str(avg_s))


Results for k = 1
Average MSEs: 54.0682
Average Stdev: 27.8439

Results for k = 2
Average MSEs: 51.6415
Average Stdev: 27.8751

Results for k = 3
Average MSEs: 52.9113
Average Stdev: 29.4842

Results for k = 4
Average MSEs: 47.8167
Average Stdev: 25.0591

Results for k = 5
Average MSEs: 52.3947
Average Stdev: 31.699

Results for k = 6
Average MSEs: 51.1038
Average Stdev: 29.4388

Results for k = 7
Average MSEs: 52.654
Average Stdev: 27.8215

Results for k = 8
Average MSEs: 50.0101
Average Stdev: 26.9064

Results for k = 9
Average MSEs: 52.9684
Average Stdev: 30.5497

Results for k = 10
Average MSEs: 54.4429
Average Stdev: 33.5244

Results for k = 11
Average MSEs: 49.0368
Average Stdev: 28.0206

Results for k = 12
Average MSEs: 52.1191
Average Stdev: 26.6053

Results for k = 13
Average MSEs: 51.387
Average Stdev: 32.0818

Results for k = 14
Average MSEs: 52.1661
Average Stdev: 28.8071

Results for k = 15
Average MSEs: 54.7486
Average Stdev: 31.3698

Results for k = 16
Average MSEs: 52.

## Problem 4: Generalization Error

Analyze and discuss the generalization error of your model with the value of k from Problem 3.

I ran the model for values of k = 1 through 19, and ran 20 iterations for each k value. I then averaged the MSEs and stdevs.

k = 12 showed the most promising results:

Average MSEs: 58.497 <- second lowest MSE

Average Stdev: 31.4172 <- lowest stdev, and second lowest is at k = 11


## Q5: Choose your own adventure

You have three options for the next part:

1. You can implement mean normalization (also called "z-score standardization") of the *features*; do not normalize the target, y. See if this improves the generalization error of your model (middle).

2. You can implement *learning curves* to see if more data would likely improve your model (easiest).

3. You can implement *weighted* kNN and use the real valued GA to choose the weights. weighted kNN assigns a weight to each item in the Euclidean distance calculation. For two points, j and k:
$$\sqrt{\sum w_i (x^k_i - x^j_i)^2}$$

You can think of normal Euclidean distance as the case where $w_i = 1$ for all features  (ambitious, but fun...you need to start EARLY because it takes a really long time to run).

The easier the adventure the more correct it must be...

<a id="normalize_data"></a>
## normalize_data

This takes the original data set and returns it as the z-score standardized version (not standardizing y / index 8).

Variables
* **to_normalize** List[List]: values from the concrete csv

**returns** List[List]: z score standardized values of the input

In [23]:
def normalize_data(to_normalize: List):
    standardized = []
    for c in range(len(to_normalize[0]) - 1):
        c_mean = sum([to_normalize[x][c] for x in range(len(to_normalize))])/len(to_normalize)
        c_stdev = statistics.stdev([to_normalize[x][c] for x in range(len(to_normalize))])
        for r in range(len(to_normalize)):
            if c == 0:
                standardized.append([])
            standardized[r].append(round((to_normalize[r][c] - c_mean)/c_stdev, 6))
    for r in range(len(to_normalize)):
        standardized[r].append(to_normalize[r][len(to_normalize[r]) - 1])
    return standardized

In [24]:
# evaluate the new z score normalized data
normalized_data = normalize_data(data)
folds = create_folds(normalized_data, 10)
evaluate_model(folds, 9, True)


Fold 0 Results:
MSE Error: 0.42948
Y (real) values:       15.82, 34.9, 58.61, 48.7, 10.54, 17.96, 26.85, 61.07, 71.3, 41.05
Yh (predicted) values: 15.52, 33.4, 59.3, 49.19, 10.54, 17.28, 26.06, 61.46, 71.3, 41.05

Fold 1 Results:
MSE Error: 0.49427
Y (real) values:       12.46, 44.09, 49.9, 18.03, 15.05, 40.29, 19.69, 23.84, 36.8, 23.22
Yh (predicted) values: 12.46, 42.42, 49.8, 18.03, 14.99, 39.0, 19.69, 23.84, 36.8, 22.53

Fold 2 Results:
MSE Error: 0.81945
Y (real) values:       46.23, 17.57, 47.13, 24.44, 55.6, 55.64, 66.1, 47.03, 12.64, 56.1
Yh (predicted) values: 45.84, 17.37, 45.71, 25.89, 56.61, 55.83, 65.2, 47.81, 11.47, 55.9

Fold 3 Results:
MSE Error: 0.73942
Y (real) values:       12.47, 40.86, 37.92, 40.86, 26.05, 29.07, 43.58, 50.24, 19.42, 40.86
Yh (predicted) values: 11.98, 42.33, 37.36, 42.33, 26.4, 29.07, 43.57, 51.04, 20.73, 40.66

Fold 4 Results:
MSE Error: 0.22763
Y (real) values:       27.04, 12.46, 36.84, 8.54, 29.65, 38.56, 24.66, 20.73, 20.73, 38.56
Yh (predic

('0.446209', '0.204976')

The results of the standardized data show the error to be significantly smaller, which definitely makes sense due to the smaller differences between raw vs normalized values.

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.